# Google Analytics 360 Sample
---

# CRISP-DM Process

Using the Cross Industry Standard Process for Data Mining, this project is an opportunity to explore and analyze data ecommerce data from Google Analytics 360 exported to BigQuery from the Google Merchandise Store.

CRISP-DM is a process model with 6 phases:

1. Busisness Understanding
2. Data Understanding
3. Data Preperation
4. Modeling
5. Evaluation
6. Deployment

## 1. Business Understanding

As an e-commerce store, much of the marketing resources are spent attracting potential customers through digital means. Using tools such as affliate marketing, display ads, and paid search terms are industry standard methods of bring new potential buyers. Our digital world is being being bifurcated into iOS and Android. Apple has recently begun to differentiate itself among the big four (Apple, Facebook, Google, Amazon) as the one who champions privacy. Safari users now see less-targeted ads, the App Store is increasing the scrutiny on how an iOS app is allowed to collect data and consumers are given easy choices, often by default, about disallowing cookies, location and other trackers.

But with those measures, are Apple users less inclined to be driven to purchase from ads?

**Business Objective**

Ultimately, this information may drive the business to adjust decisions on how or where to spend marketing dollars attracting nw customers. Our objective is to increase the effectiveness of our marketing efforts. Whether that means a reduction in spending targeted to Apple users or an increase in spending to another demographic that *is* afffected by targeted advertising.

### I will attempt to answer the following questions to meet our business objectives:

1. Do macOS/iOS affected by traffic source in the same way... organic, paid display. Are targeted ads less effective on iOS? 

2. Do macOS/iOS users purchase more than windows/android users?

3. Is there a correlation of refering traffic to higher average order value?

--- 

## 2. Data Understanding

Google Cloud has shared its Google Analytics 360 data as a [public dataset](https://cloud.google.com/bigquery/public-data/).

**Where it comes from**

The sample dataset contains obfuscated Google Analytics 360 data from the [Google Merchandise Store](https://www.googlemerchandisestore.com/shop.axd/Home?utm_source=Partners&utm_medium=affiliate&utm_campaign=Data%20Share%20Promo), a real ecommerce store. The Google Merchandise Store sells Google branded merchandise. The data is typical of what you would see for an ecommerce website. It includes the following kinds of information:

- Traffic source data: information about where website visitors originate. This includes data about organic traffic, paid search traffic, display traffic, etc.
- Content data: information about the behavior of users on the site. This includes the URLs of pages that visitors look at, how they interact with content, etc.
- Transactional data: information about the transactions that occur on the Google Merchandise Store website.

Data includes records from August of 2016 through August of 2017. 

In the next blocks of this notebook, I will collect, describe, explore, and verify our data will be useful to meet our business objectives. 


--- 

**Collect**

First, import dependancies and create Google Cloud client connection to my project through a BigQuery service account. This will give us access to make quries on the data see what is available.

Creating a connection to Google BigQuery for the first time is a non-trivial task. While there is extensive documentation from Google Cloud Products, for those unfamiliar with the process and setup this will take some time to create a project_id, a service_account and configure python libraries with the connection parameters.

Great instructions can be found [here](https://cloud.google.com/getting-started/) 

Additionally, please see the installation steps in the README file associated with this project.

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import plotly.graph_objects as go
import fohr_theme_light
import chart_studio.plotly as py
import statsmodels.api as sm
pd.options.plotting.backend = 'plotly'

# Connect to 
CREDENTIALS = service_account.Credentials.from_service_account_file('bigquery.json')
PROJECT_ID = 'udacity-bigquery'
CLIENT = bigquery.Client(credentials=CREDENTIALS, project=PROJECT_ID)

print(f'numpy = {np.__version__}')
print(f'pandas = {pd.__version__}')
print(f'bigquery = {bigquery.__version__}')

numpy = 1.19.1
pandas = 1.1.0
bigquery = 1.22.0


---

**Describe** & **Explore**

Let's take a look at what is stored in this data frame. Similar to AWS Athena, Google BigQuery has subtables inside columns. It can also auto-detect a schema on read for supported formats and allows for Nulls. 

In [2]:
query = '''
    SELECT distinct column_name, data_type
    FROM `bigquery-public-data.google_analytics_sample.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name like 'ga_sessions_%'
    ORDER BY 1
'''
data = CLIENT.query(query).result().to_dataframe()
data

,column_name,data_type
0,channelGrouping,STRING
1,clientId,STRING
2,customDimensions,"ARRAY<STRUCT<index INT64, value STRING>>"
3,date,STRING
4,device,"STRUCT<browser STRING, browserVersion STRING, ..."
5,fullVisitorId,STRING
6,geoNetwork,"STRUCT<continent STRING, subContinent STRING, ..."
7,hits,"ARRAY<STRUCT<hitNumber INT64, time INT64, hour..."
8,hits,"ARRAY<STRUCT<hitNumber INT64, time INT64, hour..."
9,socialEngagementType,STRING


**Data Pull Function** 

To make further queries of the data, build a function to reduce repetition.

In [3]:
DF = pd.DataFrame()

def get_data(query: str, client=CLIENT)-> DF:
    """
    INPUT - query  = SQL-like query string to retrieve data from Google BigQuery
            client = A google bigquery.client object set with your own credentials and project ID. 
                     Use variable named 'CLIENT' as a default or pass in your own client variable name.
    
    OUTPUT - Pandas DataFrame object
    
    Functions calls Google Cloud BigQuery and returns the results stored as a Pandas DataFrame.
    """
    return client.query(query).result().to_dataframe()


I can see that 7 of the 17 columns have a sub-structure with more data. Using a SQL-like sintax of `.` we can ultimately call these sub-column fields directly.

Let's take a look at the first one.

The `customDimensions` column has an `array` stored in each field. I can `UNNEST` the field and then use the `.` syntax to retrieve fields inside the column. In this case I will use the wildcard `*` to SELECT all the columns to view. 

In [4]:
data = get_data(''' 
    SELECT cd.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
    UNNEST(customDimensions) cd 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    ''')
data

,index,value
0,4,APAC
1,4,North America
2,4,EMEA
3,4,APAC
4,4,EMEA


Hmmm.  I don't really know what `index` means. `value` is clearly a geo region, but it looks like that might also be stored in the `GeoNetwork` column.

Let's look at another column.

In [5]:
data = get_data(''' 
    SELECT visitorId, fullVisitorId, device.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    ''')
data

,visitorId,fullVisitorId,browser,browserVersion,browserSize,operatingSystem,operatingSystemVersion,isMobile,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,javaEnabled,language,screenColors,screenResolution,deviceCategory
0,None,2558542945282276924,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
1,None,3559409406108279851,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
2,None,1236280516298011032,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
3,None,9717817368523884345,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
4,None,9422257354738249512,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop


In [6]:
data.shape[1]

19

Now that is a lot more information. The sub-fields in the `device` columns allows us to see information about the device connecting to the store. However, as this is a public dataset, much of the data has been removed. Many columns have `not available in demo dataset` as the only value in all fields. `browser`, `operatingSystem`, `isMobile` and `deviceCategory` remain. Although the last two seem to be redundant information.

---



In [7]:
data = get_data(''' 
    SELECT visitorId, fullVisitorId, geoNetwork.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    ''')
data

,visitorId,fullVisitorId,continent,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,None,4187266305458832238,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mycingular.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,None,8768848781522423530,Europe,Eastern Europe,Russia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,sknt.ru,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,None,9342546397803278307,Europe,Eastern Europe,Russia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,None,0924580349699227752,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
4,None,7272617006657421843,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset


`geoNetwork` looks to be infomation about the location of the user. Only `latitude`, `longitude` and `networkLocation` have been removed. 

---


In [8]:
data = get_data(''' 
    SELECT fullVisitorId, h.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
    UNNEST(hits) h
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 50
    ''')
data.head()

,fullVisitorId,hitNumber,time,hour,minute,isSecure,isInteraction,isEntrance,isExit,referer,...,customVariables,customDimensions,customMetrics,type,social,latencyTracking,sourcePropertyInfo,contentGroup,dataSource,publisher_infos
0,2053047893338047846,1,0,11,36,None,True,True,True,https://www.google.com/url?sa=t&rct=j&q=&esrc=...,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
1,0646625287492885645,1,0,9,54,None,True,True,True,https://www.google.it/,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
2,0811339686180783981,1,0,9,49,None,True,True,True,https://www.startpage.com/do/dsearch?query=goo...,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
3,6773336867278757036,1,0,14,6,None,True,True,True,https://www.google.com/,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]
4,8127921846898396397,1,0,13,22,None,True,True,True,https://www.google.com/,...,[],[],[],PAGE,"{'socialInteractionNetwork': None, 'socialInte...",None,None,"{'contentGroup1': '(not set)', 'contentGroup2'...",None,[]


In [9]:
data.shape[1]

34

In [10]:
data.columns

Index(['fullVisitorId', 'hitNumber', 'time', 'hour', 'minute', 'isSecure',
       'isInteraction', 'isEntrance', 'isExit', 'referer', 'page',
       'transaction', 'item', 'contentInfo', 'appInfo', 'exceptionInfo',
       'eventInfo', 'product', 'promotion', 'promotionActionInfo', 'refund',
       'eCommerceAction', 'experiment', 'publisher', 'customVariables',
       'customDimensions', 'customMetrics', 'type', 'social',
       'latencyTracking', 'sourcePropertyInfo', 'contentGroup', 'dataSource',
       'publisher_infos'],
      dtype='object')

In [11]:
data['page'].values[4]

{'pagePath': '/home',
 'hostname': 'www.googlemerchandisestore.com',
 'pageTitle': 'Google Online Store',
 'searchKeyword': None,
 'searchCategory': None,
 'pagePathLevel1': '/home',
 'pagePathLevel2': '',
 'pagePathLevel3': '',
 'pagePathLevel4': ''}

In [12]:
data['referer'].head()

0    https://www.google.com/url?sa=t&rct=j&q=&esrc=...
1                               https://www.google.it/
2    https://www.startpage.com/do/dsearch?query=goo...
3                              https://www.google.com/
4                              https://www.google.com/
Name: referer, dtype: object

34 sub-fields of data in the `hits` column. Lots of great information in this column. However, like before many pieces have been removed. 

In [13]:
data = get_data(''' 
    SELECT visitorId, fullVisitorId, totals.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 5 
    ''')
data

,visitorId,fullVisitorId,visits,hits,pageviews,timeOnSite,bounces,transactions,transactionRevenue,newVisits,screenviews,uniqueScreenviews,timeOnScreen,totalTransactionRevenue,sessionQualityDim
0,None,1824326185009798244,1,5,5,510,None,None,None,1,None,None,None,None,None
1,None,6193340579732523713,1,6,6,258,None,None,None,1,None,None,None,None,None
2,None,9710825323738535026,1,5,5,291,None,None,None,1,None,None,None,None,None
3,None,3911433614163036483,1,4,4,37,None,None,None,1,None,None,None,None,None
4,None,5051442164305284284,1,4,4,97,None,None,None,1,None,None,None,None,None


In [14]:
data.shape[1]

15

15 subfields in the `totals` column. Key here will be `transactions` and `transactionRevenue`.

In [15]:
data = get_data(''' 
    SELECT trafficSource.* 
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20160901' 
    LIMIT 50
    ''')
data.head()

,referralPath,campaign,source,medium,keyword,adContent,adwordsClickInfo,isTrueDirect,campaignCode
0,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
1,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
2,None,(not set),(direct),(none),None,None,"{'campaignId': None, 'adGroupId': None, 'creat...",True,None
3,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
4,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None


In [16]:
data.shape[1]

9

In [17]:
data['adwordsClickInfo'].values[42]

{'campaignId': None,
 'adGroupId': None,
 'creativeId': None,
 'criteriaId': None,
 'page': None,
 'slot': None,
 'criteriaParameters': 'not available in demo dataset',
 'gclId': None,
 'customerId': None,
 'adNetworkType': None,
 'targetingCriteria': None,
 'isVideoAd': None}

The `trafficSource` column also has interesting sub-fields to look at. Sadly it doesn't seem that `adwordsClickInfo` is one of them. 

---

Finally lets look at the direct columns to see the type of data involved and better understand its meaning in context. 

In [18]:
data = get_data(''' 
    SELECT  fullvisitorId, channelGrouping, clientId, socialEngagementType, userId, visitId, visitNumber, visitStartTime, visitorId	
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20170701' and '20170801'
    LIMIT 50
    ''')
data.sample(5)

,fullvisitorId,channelGrouping,clientId,socialEngagementType,userId,visitId,visitNumber,visitStartTime,visitorId
29,0044028373383778376,Organic Search,None,Not Socially Engaged,None,1500596947,1,1500596947,None
45,6431427809350567522,Organic Search,None,Not Socially Engaged,None,1500560295,1,1500560295,None
41,1693160726448147902,Organic Search,None,Not Socially Engaged,None,1500595690,1,1500595690,None
6,0702494090355446159,Organic Search,None,Not Socially Engaged,None,1500572175,1,1500572175,None
11,8387781726885662321,Organic Search,None,Not Socially Engaged,None,1500562176,1,1500562176,None


---

**Verify** 

Interestingly I saw that `fullvisitorId` number 0288998128011317800 has `7` listed in visitNumber.  It is assumned that we have seen visitor: 0288998128011317800 at least 7 times.

Let's verify the data to see if that means what I think it means. Let's pull the query back the full year and look for that `fullvisitorId` to see if we have more than one record and if the `visitNumber` increments accordingly. 

Understand that this visitor may have come to the site before the August 2016, but perhaps we can still see the visitor count increment. 

In [19]:
data = get_data(''' 
    SELECT  fullvisitorId, channelGrouping, clientId, socialEngagementType, userId, visitId, visitNumber, visitStartTime, visitorId	
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '0288998128011317800'
    LIMIT 50
    ''')
data

,fullvisitorId,channelGrouping,clientId,socialEngagementType,userId,visitId,visitNumber,visitStartTime,visitorId
0,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1492967657,4,1492967657,None
1,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1498637120,6,1498637120,None
2,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1500035030,8,1500035030,None
3,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1492085541,2,1492085541,None
4,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1494753995,5,1494753995,None
5,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1498978602,7,1498979704,None
6,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1492506599,3,1492506599,None
7,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1498978602,7,1498978602,None
8,0288998128011317800,Organic Search,None,Not Socially Engaged,None,1491825966,1,1491825966,None


Indeed we can see that this user has come to the size at least 8 times during the year we have data for. This confirms that we can use `visitNumber` in the way I assumed to help analyse the data.

---

Following this test, I will also test to be sure that dates also line up with this and look for transactionTotal amounts to determine if these are cumulative for the lifetime of the visitor.

First I will need to find a visitor that has many visit and at least a couple of trnasactions.



In [20]:
data = get_data(''' 
    SELECT  fullvisitorId, AVG(visitNumber), sum(totals.transactions), sum(totals.transactionRevenue), sum(totals.totalTransactionRevenue)
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and visitNumber > 1 and channelGrouping = 'Display'
    GROUP BY fullvisitorId
    LIMIT 50
    ''')
data.loc[data['f1_'].notna()]

,fullvisitorId,f0_,f1_,f2_,f3_


In [21]:
data = get_data(''' 
    SELECT  fullvisitorId, date, visitId, visitNumber, visitStartTime, visitorId, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '4085972867127024343'
    LIMIT 50
    ''')
data.sort_values('visitNumber')

,fullvisitorId,date,visitId,visitNumber,visitStartTime,visitorId,transactions,transactionRevenue,totalTransactionRevenue
0,4085972867127024343,20161014,1476453496,1,1476453496,None,NaN,NaN,NaN
4,4085972867127024343,20161101,1478033380,2,1478033380,None,NaN,NaN,NaN
8,4085972867127024343,20161107,1478547833,3,1478547833,None,NaN,NaN,NaN
9,4085972867127024343,20161107,1478550684,4,1478550684,None,1.0,19190000.0,26190000.0
5,4085972867127024343,20161110,1478808651,7,1478808651,None,NaN,NaN,NaN
3,4085972867127024343,20161113,1479097793,8,1479097793,None,1.0,19190000.0,24190000.0
6,4085972867127024343,20161120,1479676039,9,1479676039,None,NaN,NaN,NaN
2,4085972867127024343,20161121,1479760240,10,1479760240,None,NaN,NaN,NaN
7,4085972867127024343,20161122,1479856417,11,1479856417,None,1.0,23990000.0,29990000.0
1,4085972867127024343,20161213,1481660216,12,1481660216,None,1.0,19190000.0,22190000.0


Great. Now we can see that `totalTransactionRevenue` is not cumalitive across a visitors lifetime. In fact it looks like the value is incrementally higher than the `transactionRevenue` column. I think we can assume that a `totalTransactionRevenue` adds something like tax or shipping costs.

----

Now, can Transactions ever be more than 1 in a single record?  Meaning do transactions represent items in an order? 

In [22]:
data = get_data(''' 
    SELECT totals.transactions
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and totals.transactions > 1
    LIMIT 50
    ''')
data.head()

,transactions
0,6
1,2
2,2
3,2
4,2


Indeed they do.  

Now I know that transactions must have represent the number of items in an order. We can use this to look for visitors that purchase more than one item. 

--- 

## 3. Data Preperation 

In the next several blocks I will prepare the dataset for modeling and analysis. 

To refresh we are looking to answer these questions to solve our business objectives.

### Three Questions of the data

1. Do macOS/iOS affected by traffic source in the same way... organic, paid display. Are targeted ads less effective on iOS? 

2. Do macOS/iOS users purchase more than windows/android users?

3. Is there a correlation of refering traffic to higher average order value?



--- 

First give a sense of how much traffic is coming to the site. 

In [23]:
data = get_data(''' 
    SELECT device.operatingSystem, count(device.operatingSystem)
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    GROUP BY device.operatingSystem
    ;
    ''')

In [24]:
data = data.loc[data['f0_']>5000].sort_values('f0_', ascending=False)

In [25]:
data['f0_'] = data['f0_'].div(data['f0_'].sum(axis=0))

In [26]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        y=data['f0_'],
        x=data['operatingSystem']
    )
)
fig.update_layout(title='Traffic by Operating System')
fig.update_yaxes(tickformat=',.0%',)
fig.show(renderer='notebook_connected')

In [27]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(fig, filename = 'udacity-web-traffic', sharing='public')

In [28]:
data = get_data(''' 
    SELECT channelGrouping, device.operatingSystem
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    ;
    ''')

In [29]:
# Turn traffic source into dummy categories
data = data.drop('channelGrouping', axis=1).join(pd.get_dummies(data['channelGrouping']))
# Add total volume of traffic
totals = data.groupby('operatingSystem').sum().sum(axis=1)
# Get percentage of traffice from soruce
data = data.groupby('operatingSystem').sum().div(data.groupby('operatingSystem').sum().sum(axis=1), axis=0)
# Add totals onto DataFrame
data['totals'] = totals
# Sort Descending
data.sort_values('totals', ascending=False, inplace=True)
# Remove minor Operating Systems like Xbox and Windows Phone.
data = data.loc[data['totals']>5000]

In [30]:
data

,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,Referral,Social,totals
operatingSystem,,,,,,,,,
Windows,0.000080,0.029762,0.114999,0.003222,0.416329,0.017956,0.065587,0.352065,350072.0
Macintosh,0.000165,0.015811,0.134419,0.009372,0.315431,0.020619,0.221790,0.282392,253938.0
Android,0.000178,0.008919,0.261163,0.007781,0.506433,0.051214,0.028872,0.135441,123892.0
iOS,0.000139,0.003920,0.190991,0.006799,0.658115,0.058357,0.018734,0.062945,107665.0
Linux,0.000257,0.009391,0.250243,0.018325,0.241480,0.005652,0.376063,0.098590,35034.0
Chrome OS,0.000114,0.003455,0.190910,0.015643,0.458784,0.036602,0.255344,0.039146,26337.0


In [31]:
# Reorder columns based on volume
data = data[list(data.sum(axis=0).sort_values(ascending=False).index)]

In [32]:
refer_by_os_fig = go.Figure()

for i in data.columns[1:-2]:
    refer_by_os_fig.add_trace(
        go.Bar(
            x=data.index,
            y=data[i],
            name = i,
            hovertext=data['totals']
        )
    )
refer_by_os_fig.update_layout(title='Refering Source by Operating System')
refer_by_os_fig.update_yaxes(tickformat=',.0%',)
refer_by_os_fig.show(renderer='notebook_connected')

That's a little busy.  Let's boil this down to paid and unpaid and show just what we are looking for. 

In [33]:
data['Unpaid'] = data['Organic Search'] + data['Direct'] + data['Referral'] + data['Social']
data['Paid'] = data['Paid Search'] + data['Affiliates'] + data['Display']

<ipython-input-33-9f6d69e33808>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-9f6d69e33808>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
data

,totals,Organic Search,Direct,Social,Referral,Paid Search,Affiliates,Display,(Other),Unpaid,Paid
operatingSystem,,,,,,,,,,,
Windows,350072.0,0.416329,0.114999,0.352065,0.065587,0.017956,0.029762,0.003222,0.000080,0.948979,0.050941
Macintosh,253938.0,0.315431,0.134419,0.282392,0.221790,0.020619,0.015811,0.009372,0.000165,0.954032,0.045803
Android,123892.0,0.506433,0.261163,0.135441,0.028872,0.051214,0.008919,0.007781,0.000178,0.931908,0.067914
iOS,107665.0,0.658115,0.190991,0.062945,0.018734,0.058357,0.003920,0.006799,0.000139,0.930785,0.069075
Linux,35034.0,0.241480,0.250243,0.098590,0.376063,0.005652,0.009391,0.018325,0.000257,0.966376,0.033368
Chrome OS,26337.0,0.458784,0.190910,0.039146,0.255344,0.036602,0.003455,0.015643,0.000114,0.944185,0.055701


This looks ready to help us evaluate our business questions. Let's save this dataframe and continue preparing data for other parts of our business objectives. 

In [35]:
traffic_by_os = data.copy()

## Did visitors who came to the site via paid source have a higher rate of transaction? 

First lets check the data to test that we can see the source as a result of a transaction. The key point is to see if the data maintains the value of source even on second and third visits. Perhaps the user was brought to the site through a piad visit initially and subsequently made a purchase on another visit as a direct visitor. 

In [36]:
data = get_data(''' 
    SELECT  fullvisitorId, channelGrouping,  device.operatingSystem, date, visitId, visitNumber, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '4085972867127024343'
    LIMIT 50
    ''')
data

,fullvisitorId,channelGrouping,operatingSystem,date,visitId,visitNumber,source,medium,transactions,transactionRevenue
0,4085972867127024343,Referral,Macintosh,20161101,1478033380,2,(direct),(none),NaN,NaN
1,4085972867127024343,Referral,Macintosh,20161110,1478808651,7,(direct),(none),NaN,NaN
2,4085972867127024343,Referral,Macintosh,20161121,1479760240,10,(direct),(none),NaN,NaN
3,4085972867127024343,Referral,Macintosh,20161107,1478547833,3,(direct),(none),NaN,NaN
4,4085972867127024343,Referral,Macintosh,20161107,1478550684,4,(direct),(none),1.0,19190000.0
5,4085972867127024343,Referral,Macintosh,20161213,1481660216,12,(direct),(none),1.0,19190000.0
6,4085972867127024343,Referral,Macintosh,20161122,1479856417,11,(direct),(none),1.0,23990000.0
7,4085972867127024343,Referral,Macintosh,20161120,1479676039,9,(direct),(none),NaN,NaN
8,4085972867127024343,Referral,Macintosh,20161014,1476453496,1,(direct),(none),NaN,NaN
9,4085972867127024343,Referral,Macintosh,20161113,1479097793,8,(direct),(none),1.0,19190000.0


When looking at our case above, we can see that the channel remains the same throughout all visits. Let's check another account with transactions and multiple visits. 

In [37]:
data = get_data(''' 
    SELECT  fullvisitorId, channelGrouping, device.operatingSystem, date, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '4140559214164857738'
    LIMIT 50
    ''')

In [38]:
data

,fullvisitorId,channelGrouping,operatingSystem,date,source,medium,transactions,transactionRevenue
0,4140559214164857738,Organic Search,Macintosh,20170707,google,organic,1,243660000
1,4140559214164857738,Organic Search,Macintosh,20170606,(direct),(none),1,120250000
2,4140559214164857738,Organic Search,Macintosh,20170613,(direct),(none),1,117340000
3,4140559214164857738,Organic Search,Macintosh,20170726,google,organic,1,119400000


Again the channel grouping looks to be maintained through all visits. One more. 

In [39]:
data = get_data(''' 
    SELECT  fullvisitorId, channelGrouping, date, visitId, visitNumber, visitStartTime, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '6377171440951916612'
    LIMIT 50
    ''')
data

,fullvisitorId,channelGrouping,date,visitId,visitNumber,visitStartTime,source,medium,transactions,transactionRevenue,totalTransactionRevenue
0,6377171440951916612,Organic Search,20170716,1500238874,2,1500238874,google,organic,None,None,None
1,6377171440951916612,Organic Search,20170511,1494556236,1,1494556236,google,organic,None,None,None
2,6377171440951916612,Paid Search,20170725,1501037798,3,1501037798,google,cpc,None,None,None
3,6377171440951916612,Paid Search,20170725,1501039074,4,1501039074,google,cpc,None,None,None


Ok, Now we can see that this user has multiple types of traffic source. 

In [40]:
data = get_data(''' 
    SELECT  fullvisitorId, channelGrouping, date, visitId, visitNumber, visitStartTime, trafficSource.source, trafficSource.medium, totals.transactions, totals.transactionRevenue, totals.totalTransactionRevenue
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
    WHERE _TABLE_SUFFIX between '20160801' and '20170801' and fullvisitorId = '738777585657711314'
    LIMIT 50
    ''')
data

,fullvisitorId,channelGrouping,date,visitId,visitNumber,visitStartTime,source,medium,transactions,transactionRevenue,totalTransactionRevenue
0,738777585657711314,Display,20161219,1482183763,2,1482183763,dfa,cpm,NaN,NaN,NaN
1,738777585657711314,Display,20161228,1482955195,6,1482955195,dfa,cpm,NaN,NaN,NaN
2,738777585657711314,Display,20161220,1482262245,3,1482262245,dfa,cpm,NaN,NaN,NaN
3,738777585657711314,Display,20161220,1482267318,4,1482267318,dfa,cpm,NaN,NaN,NaN
4,738777585657711314,Display,20161220,1482272043,5,1482272043,dfa,cpm,1.0,79130000.0,86130000.0


## Before we get too far along, perhaps it would be good to see who is performing tansactions. 

Lets diferentiate between those visitors who have come to the site in any fashion through a targeted ad. 

In [41]:
data = get_data(''' 
    SELECT fullvisitorId,
           operatingSystem,
           sum(paid),
           max(visitNumber),
           sum(transactions),
           sum(transactionRevenue)/1000000
    FROM (SELECT  fullvisitorId, 
            device.operatingSystem,
            CASE WHEN channelGrouping in ('Affliates', 'Display', 'Paid Search') THEN 1 ELSE 0 end as paid, 
            visitNumber,  
            totals.transactions, 
            totals.transactionRevenue,
          FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
          -- To maintain consistency, remove inconsiquential Operating systems.
          WHERE device.operatingSystem in ('Windows', 'Macintosh', 'iOS', 'Android', 'Linux', 'Chrome OS'))
    GROUP BY 1, 2
    ''')

Aggregate this data to see the sucess rate of purchases between users who experienced at least one paid ad. 

First lets inspect the data. 

In [42]:
# Rename columns to something readable
data.columns = ['fullvisitorId', 'operatingSystem', 'Paid', 'Visits', 'Transactions', 'Revenue']
# Mark Paid column to a Boolean
data['Paid'] = data['Paid']>1

In [43]:
data.describe()

,Visits,Transactions,Revenue
count,708258.000000,10021.000000,9995.000000
mean,1.324242,1.208861,154.081525
std,1.977412,0.976053,903.380765
min,1.000000,1.000000,0.010000
25%,1.000000,1.000000,24.485000
50%,1.000000,1.000000,48.890000
75%,1.000000,1.000000,108.470000
max,395.000000,36.000000,77113.430000


Looks like there are two a big outliers in there.  Before Moving on, we should remove those. 

In [44]:
data.loc[data['Revenue'] > data['Revenue'].quantile(.99)]

,fullvisitorId,operatingSystem,Paid,Visits,Transactions,Revenue
6524,0220695720492664721,Macintosh,True,16,7.0,2224.52
41871,3244885836845029978,Macintosh,False,5,4.0,4319.52
59217,428994201200499894,Macintosh,True,26,3.0,3286.57
59427,4604965471651937146,Chrome OS,False,3,1.0,4280.70
60324,6147396474895233852,Chrome OS,False,10,5.0,3265.28
...,...,...,...,...,...,...
649812,4950411203281265700,Android,False,10,3.0,2250.25
670720,2446685875964479851,Macintosh,True,11,6.0,4160.76
671649,3921649958751416379,Chrome OS,True,15,6.0,3142.20
689905,7463172420271311409,Chrome OS,False,18,9.0,7225.10


In [45]:
data = data.loc[(data['Revenue'] < data['Revenue'].quantile(.99)) | data['Revenue'].isna()]

Let's check vor Outliers in `Visits` as well. 

In [46]:
data.loc[data['Visits'] > data['Visits'].quantile(.999)]

,fullvisitorId,operatingSystem,Paid,Visits,Transactions,Revenue
59,9138278838478762048,Macintosh,False,30,NaN,NaN
116,5112369122544987822,Chrome OS,False,20,5.0,1230.84
181,7117116572860849022,Macintosh,True,30,NaN,NaN
260,603373162136570069,Windows,False,44,NaN,NaN
262,0769560476351515188,Windows,False,68,NaN,NaN
...,...,...,...,...,...,...
690071,9467619663210756890,Windows,False,41,2.0,335.58
690457,5101701802972301118,Macintosh,False,24,NaN,NaN
690723,034879850858322847,Macintosh,False,30,NaN,NaN
692097,8386662856094972129,Macintosh,False,33,NaN,NaN


In [47]:
data = data.loc[(data['Visits'] < data['Visits'].quantile(.999)) | (data['Visits'].isna())]

In [48]:
len(data)

707448

Now Let's aggregate the data to look for any patterns insights in Paid traffic.

In [49]:
# Group data by operating system and the paid boolean. 
agg_data = data.groupby(['operatingSystem', 'Paid']).agg({'fullvisitorId':'count', 'Visits':'sum', 'Transactions':'count', 'Revenue':['sum', 'sem']})

In [50]:
agg_data.columns = ['fullvisitorId',
                   'Visits',
             'Transactions',
                  'Revenue',
                  'Revenue_SEM']

In [51]:
# Calculate the sucess rate of Unique visitors to transactions 
agg_data['Rate'] = agg_data['Transactions']/agg_data['Visits']
# Create Average Order size from Revenue and Transactions
agg_data['Ave_Order'] = round((agg_data['Revenue']/agg_data['Transactions']), 2)
# Reformat Revenue to readable value
agg_data['Revenue'] = round(agg_data['Revenue'], 2)
agg_data['Revenue_ME'] = agg_data['Revenue_SEM']*1.96

In [52]:
agg_data.reset_index(inplace=True)

In [53]:
agg_data

,operatingSystem,Paid,fullvisitorId,Visits,Transactions,Revenue,Revenue_SEM,Rate,Ave_Order,Revenue_ME
0,Android,False,97279,119417,375,23277.98,6.903727,0.003140,62.07,13.531306
1,Android,True,1243,4547,53,4649.79,21.142060,0.011656,87.73,41.438437
2,Chrome OS,False,17202,26066,701,96476.87,8.422773,0.026893,137.63,16.508636
3,Chrome OS,True,192,800,32,8444.49,65.765738,0.040000,263.89,128.900846
4,Linux,False,25168,37552,717,42211.27,2.405015,0.019094,58.87,4.713830
5,Linux,True,120,527,12,735.98,11.298084,0.022770,61.33,22.144244
6,Macintosh,False,186688,260127,5235,642241.70,2.657324,0.020125,122.68,5.208355
7,Macintosh,True,1008,4476,190,26151.80,16.154638,0.042449,137.64,31.663091
8,Windows,False,292779,344782,1848,180517.88,4.286546,0.005360,97.68,8.401630
9,Windows,True,778,3028,77,7186.79,15.581563,0.025429,93.33,30.539864


Determine Confidence Interval

In [54]:
def get_lo_ci(row):
    return sm.stats.proportion_confint(count = (row['Visits'] * row['Rate']), nobs=row['Visits'], method='normal')[0]
def get_hi_ci(row):
    return sm.stats.proportion_confint(count = (row['Visits'] * row['Rate']), nobs=row['Visits'], method='normal')[1]

In [55]:
agg_data['lo_ci'] = agg_data.apply(get_lo_ci, axis=1)
agg_data['hi_ci'] = agg_data.apply(get_hi_ci, axis=1)
agg_data['RATE_ME'] = agg_data['Rate'] - agg_data['lo_ci']

In [56]:
agg_data

,operatingSystem,Paid,fullvisitorId,Visits,Transactions,Revenue,Revenue_SEM,Rate,Ave_Order,Revenue_ME,lo_ci,hi_ci,RATE_ME
0,Android,False,97279,119417,375,23277.98,6.903727,0.003140,62.07,13.531306,0.002823,0.003458,0.000317
1,Android,True,1243,4547,53,4649.79,21.142060,0.011656,87.73,41.438437,0.008536,0.014776,0.003120
2,Chrome OS,False,17202,26066,701,96476.87,8.422773,0.026893,137.63,16.508636,0.024929,0.028857,0.001964
3,Chrome OS,True,192,800,32,8444.49,65.765738,0.040000,263.89,128.900846,0.026421,0.053579,0.013579
4,Linux,False,25168,37552,717,42211.27,2.405015,0.019094,58.87,4.713830,0.017709,0.020478,0.001384
5,Linux,True,120,527,12,735.98,11.298084,0.022770,61.33,22.144244,0.010035,0.035506,0.012736
6,Macintosh,False,186688,260127,5235,642241.70,2.657324,0.020125,122.68,5.208355,0.019585,0.020664,0.000540
7,Macintosh,True,1008,4476,190,26151.80,16.154638,0.042449,137.64,31.663091,0.036542,0.048355,0.005906
8,Windows,False,292779,344782,1848,180517.88,4.286546,0.005360,97.68,8.401630,0.005116,0.005604,0.000244
9,Windows,True,778,3028,77,7186.79,15.581563,0.025429,93.33,30.539864,0.019822,0.031037,0.005607


---

## 4. Model

The questions we need to find in our business objectives may not need algorithmic modeling. If we were looking to find the variable that was more correlated to purchases or perhaps use this data mining to forecast future trends, we may need to add more. Given our question is only to determine the effectiveness of advertising to a group we can simply look at the historical data. 

However, we should check the results with confidence intervals of 95% to verify we are seeing differences outside of random chance. 

--- 

## 5. Evaluation

Now that I have prepared the data and added some basic modeling to help meet our business objectives, lets evaluate the results and see if we can learn what we need to from the data. 

--- 

**Traffic by Operating System**


In [57]:
traffic_fig = go.Figure()
traffic_fig.add_trace(
    go.Bar(
        x=traffic_by_os.index,
        y=traffic_by_os['Paid']
    )
)
traffic_fig.update_layout(title='Paid Traffic by Operating System')
traffic_fig.update_yaxes(tickformat=',.0%',)
traffic_fig.show(renderer='notebook_connected')


**Review Process**

Interesting to see that both mobile operating systems are being driven to the site at a higher rate from paid sources.
One major flaw of this analysis is that the volume of impressions is not known. The Google Merchandise Store may have purchased more advertisements targeted at mobile device users resulting in higher traffic from those Operating Systems. However, we can see that Android and iOS users are coming to the site from paid ads at nearly the same rate. Again, it is possible that the Google Merchandise Store purchased more targeted ad impressions to one group or another in order to realize the same site traffic volume. In a real world scenario working inside an organization, the effectiveness of the impressions converting to traffic would be a obvious metric to track.

---

**Conversion Rate By Operation System**

In [59]:
conversion_by_os_fig = go.Figure()
conversion_by_os_fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==True]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==True]['Rate'],
        error_y=dict(type='data', array=agg_data.loc[agg_data['Paid']==True]['RATE_ME']),
        name="From Paid Source"
    )
)
conversion_by_os_fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==False]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==False]['Rate'],
        error_y=dict(type='data', array=agg_data.loc[agg_data['Paid']==False]['RATE_ME']),
        name="From Organic Source"
    )
)
conversion_by_os_fig.update_layout(title='Conversion Rate by Operating System')
conversion_by_os_fig.update_yaxes(tickformat='%')
conversion_by_os_fig.show(renderer='notebook_connected')

**Review Process**

It looks clear that with each operating system we can see an increase in conversion for the users who had at least one of their visits from a paid source.
Further, macOS desktop users had the best conversion from paid traffic. Clearly the targeted ads, and paid search is still effective for Apple users in 2016–2017.

One note is that Linux and ChromeOS did not produce enough data points to reject the null hypothesis. The increase in either rate of transactions could be the result of sampling error. 

Now lets loot at Average order size. 

In [60]:
ave_order_fig = go.Figure()
ave_order_fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==True]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==True]['Ave_Order'],
        error_y=dict(type='data', array=agg_data.loc[agg_data['Paid']==True]['Revenue_ME']),
        name="From Paid Source"
    )
)
ave_order_fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==False]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==False]['Ave_Order'],
        error_y=dict(type='data', array=agg_data.loc[agg_data['Paid']==False]['Revenue_ME']),
        name="From Organic Source"
    )
)
ave_order_fig.update_layout(title='Average Order by Operating System')
ave_order_fig.update_yaxes(tickformat='$')
ave_order_fig.show(renderer='notebook_connected')

**Review Process**

In most cases we can see a very small difference between paid traffic and organic traffic regarding average order size.

It doesn’t seem to be the case that macOS users had the highest order value. Instead we can see that ChomeOS users coming from a paid source purchased more goods per transaction, over $200. Logically, it makes sense that ChromeOS users would also be fans of Google branded products, However we are already aware that the volume of ChromeOS data was also much lower than others creating a higher margin of error. 

--- 

Let's blow the whole thing up. We may have seen paid traffic increase conversion. We also saw paid traffic not affect average ordersize. However, what about raw volume? 

In [61]:
total_rev_fig = go.Figure()
total_rev_fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==True]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==True]['Revenue'],
        name="From Paid Source"
    )
)
total_rev_fig.add_trace(
    go.Bar(
        x=agg_data.loc[agg_data['Paid']==False]['operatingSystem'],
        y=agg_data.loc[agg_data['Paid']==False]['Revenue'],
        name="From Organic Source"
    )
)
total_rev_fig.update_layout(title='Total Revenue by Operating System')
total_rev_fig.update_yaxes(tickformat='$')
total_rev_fig.show(renderer='notebook_connected')

**Review Process**

It looks as though organic macOS users are dominating sales revenue. This surely forces the business objectives to be put into perspective. 

--- 

### Thoughts

It’s very important to remember that this data is now nearly 5 years old. A lot has happened in the world of digital advertising, mobile purchasing and the change in marketshare of the OS leaders. While it is interesting to be able to explore and analyze free public data from Google Analytics 360 on BigQuery, the missing pieces regarding original ad impressions and the specifics of purchase data keep this analysis from becoming truly useful.

That said, regarding the question of effectiveness of digital advertising on Apple users, it looks as though Display Ads, Affiliates, and Paid Search are just as effective at driving traffic, converting to sales and increasing order size for Apple users.

We saw conversion rate nearly double for all OS groups when coming from paid sources. And, percentage of traffic from paid sources was very similar across most of the OS options. Therefore, no OS could be correlated with less traffic (assuming the original ad-buy was similar).

However, in the end, product is king. The biggest source of revenue found the products organically and from their macOS equipped machine. For all the money and effort put into digital advertising in this case we are not able to directly relate these purchases to the investment. We do learn, however, which group is most likely to buy. Perhaps we can use this information to help plan product development and ad buys in the future.


## 6. Deployment 

In the final step. I will use Plotly's Chart Studio to publish the visualizations in this notebook. 

This will allow for the full functionality of plotly's visualizations including Hovertext, image capture and zoom. 

I will then embed these reactive visuals in a Medium post to share the results. 

In [62]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(refer_by_os_fig, filename = 'udacity-web-traffic_by_source', sharing='public')

In [63]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(traffic_fig, filename = 'udacity-paid_web-traffic', sharing='public')

In [64]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(conversion_by_os_fig, filename = 'udacity-conversion_by_os', sharing='public')

In [65]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(ave_order_fig, filename = 'udacity_aveOrder_by_os', sharing='public')

In [66]:
# Send Figure to Plotly Chart Studio to share on embed
# py.plot(total_rev_fig, filename = 'udacity-totRev_by_os', sharing='public')